In [85]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import timeit

In [33]:
def scrape(mainUrl):
    fromDates = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
    toDates = ['2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
    pomUrl = mainUrl + "ALK"
    response = requests.get(pomUrl)
    soup = BeautifulSoup(response.text, 'html.parser')
    sifri = soup.select("#Code > option")
    companies = []
    for s in sifri:
      if not any(char.isdigit() for char in s.text):
        companies.append(s.text)
    dataFrameList = []
    for j in range(len(companies)):
        myList = []
        for i in range(len(fromDates)):
            url = mainUrl + companies[j] + "?FromDate=10.11." + fromDates[i] + "&ToDate=09.11." + toDates[i]
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            rows = soup.select("#resultsTable > tbody > tr")
            for row in rows:
                datum = row.select_one("td:nth-child(1)").text
                cena_posledna = row.select_one("td:nth-child(2)").text
                mak = row.select_one("td:nth-child(3)").text
                minimum = row.select_one("td:nth-child(4)").text
                cena_prosecna = row.select_one("td:nth-child(5)").text
                procent_promet = row.select_one("td:nth-child(6)").text
                kolicina = row.select_one("td:nth-child(7)").text
                BEST_MKD = row.select_one("td:nth-child(8)").text
                vk_promet = row.select_one("td:nth-child(9)").text
        
                cena_dict = {
                    "Date": datum,
                    "Price": cena_posledna,
                    "Maximum": mak,
                    "Minimum": minimum,
                    "Average Price": cena_prosecna,
                    "Procent of turnover": procent_promet,
                    "Quantity": kolicina,
                    "BEST MKD": BEST_MKD,
                    "Total turnover": vk_promet
                }
                myList.append(cena_dict)
        df = pd.DataFrame(myList)
        dataFrameList.append(df)
            
    return dataFrameList

In [35]:
dataFor1Year = scrape("https://www.mse.mk/mk/stats/symbolhistory/")

In [83]:
for i in range(len(dataFor1Year)):
    pomosna = dataFor1Year[i]
    pomosna['Date'] = pd.to_datetime(pomosna['Date'], format='%d.%m.%Y')
    df_sorted = pomosna.sort_values(by='Date')
    df_sorted.to_csv('tables for Domasna1/table' + str(i) + '.csv')

KeyError: 'Date'

In [57]:
pomosna['Date'] = pd.to_datetime(dataFor1Year[1]['Date'], format='%d.%m.%Y')
df_sorted = pomosna.sort_values(by='Date')

In [59]:
df_sorted

,Date,Price,Maximum,Minimum,Average Price,Procent of turnover,Quantity,BEST MKD,Total turnover
242,2014-11-10,"4.850,00","4.861,00","4.850,00","4.853,88","-0,08",123,597.027,597.027
241,2014-11-11,"4.850,00","4.852,00","4.800,00","4.836,46","-0,36",242,1.170.424,1.170.424
240,2014-11-12,"4.900,00","4.900,00","4.850,00","4.887,66","1,06",314,1.534.725,1.534.725
239,2014-11-13,"4.860,00","4.860,00","4.850,00","4.856,25","-0,64",8,38.850,38.850
238,2014-11-14,"4.949,00","4.949,00","4.860,00","4.908,29","1,07",35,171.790,171.790
...,...,...,...,...,...,...,...,...,...
2213,2024-11-04,"22.300,00","22.300,00","22.250,00","22.293,98","-0,10",408,9.095.944,9.095.944
2212,2024-11-05,"22.300,00","22.300,00","22.298,00","22.299,74","0,03",3.434,76.577.300,76.577.300
2211,2024-11-06,"22.500,00","22.502,00","22.301,00","22.484,06","0,83",187,4.204.519,4.204.519
2210,2024-11-07,"23.000,00","23.000,00","22.990,00","22.997,62","2,28",130,2.989.691,2.989.691
